In [6]:
import pandas as pd

In [7]:
thresh1 = 0.6533
thresh2 = 0.57
global_thresh1 = 0.49
global_thresh2 = 0.55
global_threshold = 0.6*global_thresh1 + 0.4*global_thresh2
thresh = 0.6*thresh1 + 0.4*thresh2
print(f"Global threshold: {global_threshold}")
print(f"Threshold: {thresh}")

Global threshold: 0.514
Threshold: 0.61998


In [ ]:
df_A = pd.read_csv('result_b7+bert.csv')
df_B = pd.read_csv('result_git2.csv')
df_merge = pd.merge(df_A, df_B, on='id', suffixes=('_A', '_B'))
df_merge = df_merge.rename(columns={'label_A': 'B7+Deberta', 'label_B': 'Git-2', 'probability_A': 'B7+Deberta_proba', 'probability_B': 'Git-2_proba'})
df_merge['vote'] = df_merge['B7+Deberta'] + df_merge['Git-2']
df_merge.head(10)

,id,B7+Deberta,B7+Deberta_proba,Git-2,Git-2_proba,vote
0,100167480,1,0.999535,1,0.998678,2
1,162710660,1,0.999271,1,0.998109,2
2,2538518632,1,0.999251,0,0.555544,1
3,5728197485,1,0.999249,1,0.996125,2
4,8550761069,1,0.999218,1,0.997798,2
5,520466994,1,0.999124,1,0.842766,2
6,865791869,1,0.999120,1,0.998773,2
7,12454037345,1,0.999085,1,0.999685,2
8,7896552942,1,0.999065,1,0.976442,2
9,2212959658,1,0.999048,1,0.995700,2


# Weight

In [9]:
df_merge['final_proba'] = 0.6*df_merge['B7+Deberta_proba'] + 0.4*df_merge['Git-2_proba']
df_merge['label'] = df_merge.apply(lambda row: 1 if row['final_proba'] >= thresh else 0, axis=1)
df_merge['global_label'] = df_merge.apply(lambda row: 1 if row['final_proba'] >= global_threshold else 0, axis=1)
df_merge.sort_values(by='final_proba', ascending=False, inplace=True)
df_merge.to_csv('result_merge.csv', index=False)
df_merge.head(473)

,id,B7+Deberta,B7+Deberta_proba,Git-2,Git-2_proba,vote,final_proba,label,global_label
7,12454037345,1,0.999085,1,0.999685,2,0.999325,1,1
0,100167480,1,0.999535,1,0.998678,2,0.999192,1,1
29,8659894365,1,0.998770,1,0.999735,2,0.999156,1,1
13,2856170843,1,0.999032,1,0.999288,2,0.999134,1,1
20,2406862400,1,0.998913,1,0.999358,2,0.999091,1,1
...,...,...,...,...,...,...,...,...,...
417,6094140818,1,0.916678,0,0.215063,1,0.636032,1,1
409,2532347594,1,0.925211,0,0.180053,1,0.627148,1,1
444,317821095,1,0.770193,0,0.410610,1,0.626360,1,1
394,4826710094,1,0.941388,0,0.146868,1,0.623580,1,1


In [10]:
num_positive = df_merge['label'].sum()
num_global_positive = df_merge['global_label'].sum()
print(f"Number of positive labels: {num_positive}")
print(f"Number of global positive labels: {num_global_positive}")
sum_vote_thresh = ((2*num_positive) - (df_merge[df_merge['label'] == 1]['vote'].sum()))/2
print(f"Sum of votes difference: {sum_vote_thresh}")
sum_vote_thresh_global = ((2*num_global_positive) - (df_merge[df_merge['label'] == 1]['vote'].sum()))/2
print(f"Sum of global votes difference: {sum_vote_thresh_global}")

Number of positive labels: 472
Number of global positive labels: 492
Sum of votes difference: 26.0
Sum of global votes difference: 46.0


In [11]:
df_submission_weight = df_merge[['id', 'label']]
df_submission_weight.to_csv('submission_weight.csv', index=False)
df_submission_weight.head()

,id,label
7,12454037345,1
0,100167480,1
29,8659894365,1
13,2856170843,1
20,2406862400,1


# Ranking


In [ ]:
from scipy.stats import rankdata
import pandas as pd

df1 = pd.read_csv("result_b7+bert.csv")
df2 = pd.read_csv("result_git2.csv")

merged = df1.merge(df2, on='id', suffixes=('_b7', '_git2'))

merged['rank_b7'] = rankdata(merged['probability_b7'])
merged['rank_git2'] = rankdata(merged['probability_git2'])
merged['avg_rank'] = (merged['rank_b7'] + merged['rank_git2']) / 2

merged = merged.sort_values(by='avg_rank', ascending=False)
merged['label'] = 0
merged.iloc[:467, merged.columns.get_loc('label')] = 1

merged.head(468)

,id,label_b7,probability_b7,label_git2,probability_git2,rank_b7,rank_git2,avg_rank,label
7,12454037345,1,0.999085,1,0.999685,1313.0,1317.0,1315.0,1
13,2856170843,1,0.999032,1,0.999288,1307.0,1306.0,1306.5,1
20,2406862400,1,0.998913,1,0.999358,1300.0,1312.0,1306.0,1
29,8659894365,1,0.998770,1,0.999735,1291.0,1318.0,1304.5,1
21,2861309138,1,0.998888,1,0.999240,1299.0,1304.0,1301.5,1
...,...,...,...,...,...,...,...,...,...
353,3272649154,1,0.973797,0,0.249708,967.0,748.0,857.5,1
592,8142499749,0,0.097784,1,0.960069,728.0,987.0,857.5,1
448,9048203603,1,0.760119,0,0.524592,872.0,838.0,855.0,1
472,9647993306,0,0.641298,1,0.640358,848.0,861.0,854.5,1


In [13]:
submission_rank = merged[['id', 'label']]
submission_rank.to_csv('submission_rank.csv', index=False)

# Check

In [ ]:
import pandas as pd


# Chỉ giữ lại các ID có Label == 1
df1_pos = df1[df1['label'] == 1].copy()
df2_pos = df2[df2['label'] == 1].copy()

# Reset index để so sánh từng dòng
df1_pos = df1_pos.reset_index(drop=True)
df2_pos = df2_pos.reset_index(drop=True)

# Tổng số mẫu label == 1
print(f"Tổng label == 1 trong df1: {len(df1_pos)}")
print(f"Tổng label == 1 trong df2: {len(df2_pos)}")

# So sánh vị trí và ID giống nhau
same_order = sum(id1 == id2 for id1, id2 in zip(df1_pos['id'], df2_pos['id']))
print(f"Số lượng id giống nhau và đúng thứ tự: {same_order} / {min(len(df1_pos), len(df2_pos))}")

# So sánh không xét thứ tự
set1 = set(df1_pos['id'])
set2 = set(df2_pos['id'])

same_ids = set1.intersection(set2)
diff_ids = set1.symmetric_difference(set2)

print(f"Số lượng id giống nhau (không xét thứ tự): {len(same_ids)}")
print(f"Số lượng id KHÁC nhau (ở label == 1): {len(diff_ids)}")

Tổng label == 1 trong df1: 469
Tổng label == 1 trong df2: 470
Số lượng id giống nhau và đúng thứ tự: 0 / 469
Số lượng id giống nhau (không xét thứ tự): 420
Số lượng id KHÁC nhau (ở label == 1): 99
